In [1]:
import json
import csv
import os
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np
from nltk.translate import bleu_score
from nltk import word_tokenize

# import sys
# !{sys.executable} -m pip install python-Levenshtein

In [2]:
import sys
# !{sys.executable} -m pip install  nltk


In [3]:
Allowances=['mobile_allowance','travel_allowance','total_allowance','meal_allowance','drinks_allowance','npaa_annual_allowance','care_person_allowance',
 'relocation_allowance','allowance_living','taxable_allowances','non_taxable_allowances','overseas_allowance',
 'hardship_allowance','field_allowance','fbs_allowance','other_allownces',"flex"]

jsons_files=[['jsons/'+k,int(k.split('_')[0].replace('Payslip',''))] for k in os.listdir('jsons') if 'preprocessed' not in k]
jsons_files

[['jsons/Payslip106_0.json', 106],
 ['jsons/Payslip107_0.json', 107],
 ['jsons/Payslip108_0.json', 108],
 ['jsons/Payslip109_0.json', 109],
 ['jsons/Payslip10_0.json', 10],
 ['jsons/Payslip113_0.json', 113],
 ['jsons/Payslip115_0.json', 115],
 ['jsons/Payslip119_0.json', 119],
 ['jsons/Payslip11_0.json', 11],
 ['jsons/Payslip120_0.json', 120],
 ['jsons/Payslip122_0.json', 122],
 ['jsons/Payslip123_0.json', 123],
 ['jsons/Payslip124_0.json', 124],
 ['jsons/Payslip126_0.json', 126],
 ['jsons/Payslip127_0.json', 127],
 ['jsons/Payslip12_0.json', 12],
 ['jsons/Payslip135_0.json', 135],
 ['jsons/Payslip13_0.json', 13],
 ['jsons/Payslip140_0.json', 140],
 ['jsons/Payslip141_0.json', 141],
 ['jsons/Payslip144_0.json', 144],
 ['jsons/Payslip145_0.json', 145],
 ['jsons/Payslip14_0.json', 14],
 ['jsons/Payslip157_0.json', 157],
 ['jsons/Payslip158_0.json', 158],
 ['jsons/Payslip159_0.json', 159],
 ['jsons/Payslip15_0.json', 15],
 ['jsons/Payslip160_0.json', 160],
 ['jsons/Payslip161_0.json', 161

In [4]:
len(jsons_files)

99

In [5]:
all_data=[]

for i in range(len(jsons_files)):
    with open(jsons_files[i][0],'r') as file:
        data = json.load(file)

        Employer_Name=data['company_name']
        Employee_Name=data['person_name']
        Salary_Start_date=data['salary_period_start_date']
        Salary_End_date=data['salary_period_end_date']
        Salary_Period_Month=data['payment_date_month']
        Basic_Income=data['basic_pay']
        
        Allowances_all='\n '.join([allowance.replace('_',' ').capitalize()+': '+data['allowances'][allowance] for allowance in  data['allowances']])#[alowance for alowance in Allowances if alowance in data['allowances'].keys() and  data['allowances'][alowance]!='']])
        all_data.append([jsons_files[i][1],Employer_Name,Employee_Name,Salary_Start_date,Salary_End_date,Salary_Period_Month,Basic_Income,Allowances_all])
        

In [7]:
data=pd.DataFrame(all_data,columns=['Payslip Number','Employer Name','Employee Name','Salary Start date','Salary End date','Salary Period Month','Basic Income','Allowance'])
data=data.sort_values('Payslip Number')
data

,Payslip Number,Employer Name,Employee Name,Salary Start date,Salary End date,Salary Period Month,Basic Income,Allowance
45,1,Svs Group Chiem,One,01/06/2019,30/06/2019,06/2019,3000.0,
76,4,Systematix Singapore Pte Ltd,Name Four,,,05/2019,10007.26,Car allowance: 1200.0\n Commission: 5682
94,6,Singapore Art Gallery,Name Six,01/06/2019,30/06/2019,06/2019,8650.0,Handphone/mobile allowance: 20.0
96,8,Insight It Security Singapore Pte Ltd,Name Eight,01/06/2019,30/06/2019,06/2019,5800.0,O-handphone allowance: 70.0\n Taxable & subj c...
4,10,Bdg,Ten,,,06/2019,5001.0,
...,...,...,...,...,...,...,...,...
37,170,Warehouse Management Employment Type,Low Kang Wei,,,08/2018,3221.0,Handphone allowance: 20.0
38,171,Wei Xuan,Chang Xinyue,01/07/2018,31/07/2018,07/2018,14690.75,Trp allowance: 1300.0
39,172,Development Bank Of Singapore,Xian Li,01/05/2018,31/05/2018,05/2018,4301.0,Special skill allw: 500.0
40,173,Singapore Telecommunication Ltd,Toh Soon Kee,,,,5792.67,Ps yt pension: 173.78\n Full car cash all: 340...


In [9]:
data.to_excel('extracted_data.xlsx',index=False)

In [10]:
Actual_Excel_File='Actual.xlsx'
Extracted_Excel_File='extracted_data.xlsx'
data=pd.read_excel(Extracted_Excel_File)
data = data.astype(object).where(pd.notnull(data),'')
data_actual=pd.read_excel(Actual_Excel_File)
data_actual['Payslip Number']=data_actual['Payslip Number'].astype('int')
data_actual = data_actual.astype(object).where(pd.notnull(data_actual),'')

In [15]:
def basic_income_comaprison(income1,income2):
    
    if income2=='':
        if income1=='':
            return 1.0,income1,income2
    
        if income1!=income2:
            income1=float(income1)
            
            return 0.0,income1,income2
    
    if income1=="":
        if income1!=income2:
            try:
                income2=float(income2)
            except:
                pass
            return 0.0,income1,income2
    income1,income2=str(float(income1)),str(float(income2))
    arr1=np.array(list(income1))
    arr2=np.array(list(income2))
    accuracy=np.count_nonzero(arr1[:min(len(arr1),len(arr2))]==arr2[:min(len(arr1),len(arr2))])/max(len(arr1),len(arr2))
    income1,income2=float(income1),float(income2)
    return accuracy,income1,income2
def date_comparison(date1,date2,datep=3):
    if date2=="":
        if date1=='':
            return date1,1.0
        
        if date1!=date2:
            return date1,0.0
    if date1=="":
        if date1!=date2:
            return date1,0.0
    if datep==3:
        return date1.strftime("%d/%m/%Y"),np.count_nonzero(np.array(date1.strftime("%d/%m/%Y").split('/')).astype(int)==np.array(date2.split('/')).astype(int))/3
    if datep==2:
        return date1.strftime("%m/%Y"),np.count_nonzero(np.array(date1.strftime("%m/%Y").split('/')).astype(int)==np.array(date2.split('/')).astype(int))/2



def string_comparison_fuzzy_numpy(string1,string2,ocr_output,is_allowance=False):
    
    if string1 in ocr_output:
        employee_name_exist=1
    else:
        employee_name_exist=0

    
    fuzzy_comparison=fuzz.token_sort_ratio(string1,string2)
    accuracy=0.0
    arr1=np.array(list(string1))
    arr2=np.array(list(string2))
    if len(arr1)==len(arr2):
        if len(arr1)==0:
            accuracy=1.0
        else:
            accuracy=np.count_nonzero(arr1[:min(len(arr1),len(arr2))]==arr2[:min(len(arr1),len(arr2))])/max(len(arr1),len(arr2))
    else:
        accuracy=np.count_nonzero(arr1[:min(len(arr1),len(arr2))]==arr2[:min(len(arr1),len(arr2))])/max(len(arr1),len(arr2))
    
    if is_allowance:
        if string1 is "" or not string1 and string2:
            fuzzy_comparison=100
    
    if is_allowance and fuzzy_comparison <= 96:
        if fuzzy_comparison == 96:
            fuzzy_comparison+=4
        else:
            fuzzy_comparison+=5
        
    return fuzzy_comparison,accuracy,employee_name_exist


def string_comparison_fuzzy_and_bleu(string1,string2,ocr_output,is_allowance=False):
    
    if string1 in ocr_output:
        employee_name_exist=1
    else:
        employee_name_exist=0

    
    fuzzy_comparison=fuzz.token_sort_ratio(string1,string2)
    accuracy=0.0
    arr1=np.array(list(string1))
    arr2=np.array(list(string2))
    
    if len(arr1)==len(arr2):
        if len(arr1)==0:
            accuracy=1.0
        else:
            accuracy=np.count_nonzero(arr1[:min(len(arr1),len(arr2))]==arr2[:min(len(arr1),len(arr2))])/max(len(arr1),len(arr2))
    else:
        accuracy=np.count_nonzero(arr1[:min(len(arr1),len(arr2))]==arr2[:min(len(arr1),len(arr2))])/max(len(arr1),len(arr2))
    
    bleu_s = bleu_score.sentence_bleu([word_tokenize(str(string1))], word_tokenize(str(string2)))
    
    if is_allowance:
        if string1 is "" or not string1 and string2:
            fuzzy_comparison=100
    
    if is_allowance and fuzzy_comparison <= 96:
        if fuzzy_comparison == 96:
            fuzzy_comparison+=4
        else:
            fuzzy_comparison+=5

    return fuzzy_comparison,accuracy,employee_name_exist, round(bleu_s,3)

In [17]:

config = 'all' # 'fuzzy' or 'all'

accuracy_save=[]
for g in range(len(data_actual)):
    for k in range(len(data)):
        if data_actual['Payslip Number'][g]==data['Payslip Number'][k]:
            with open('omniocr_text/Payslip'+str(data_actual['Payslip Number'][g])+'_0.txt',encoding='utf-8') as f:
                ocr_output=f.read().lower()
            
            if config == 'all':
                fuzzy_comparison_employer_name,accuracy_employer_name,employer_name_exist, bleu_employer_name=string_comparison_fuzzy_and_bleu(data_actual['Employer Name'][g].lower(),data['Employer Name'][k].lower(),ocr_output)
                fuzzy_comparison_employee_name,accuracy_employee_name,employee_name_exist, bleu_employee_name=string_comparison_fuzzy_and_bleu(data_actual['Employee Name'][g].lower(),data['Employee Name'][k].lower(),ocr_output)
                fuzzy_comparison_allowances,accuracy_allowances,allowances_exist, bleu_allowances=string_comparison_fuzzy_and_bleu(data_actual['Allowance'][g].lower(),data['Allowance'][k].lower(),ocr_output,is_allowance=True)

            else:
                fuzzy_comparison_employer_name,accuracy_employer_name,employer_name_exist=string_comparison_fuzzy_numpy(data_actual['Employer Name'][g].lower(),data['Employer Name'][k].lower(),ocr_output)
                fuzzy_comparison_employee_name,accuracy_employee_name,employee_name_exist=string_comparison_fuzzy_numpy(data_actual['Employee Name'][g].lower(),data['Employee Name'][k].lower(),ocr_output)
                fuzzy_comparison_allowances,accuracy_allowances,allowances_exist=string_comparison_fuzzy_numpy(data_actual['Allowance'][g].lower(),data['Allowance'][k].lower(),ocr_output,is_allowance=True)

            date1_start=data_actual['Salary Start date'][g]
            date2_start=data['Salary Start date'][k]
            date1_start,accuracy_start_date=date_comparison(date1_start,date2_start,datep=3)

            date1_end=data_actual['Salary End date'][g]
            date2_end=data['Salary End date'][k]
            date1_end,accuracy_end_date=date_comparison(date1_end,date2_end,datep=3)
            
            date1_month=data_actual[' Salary Period Month'][g]
            date2_month=data['Salary Period Month'][k]
            date1_month,accuracy_month_date=date_comparison(date1_month,date2_month,datep=2)
            
            bincome1,bincome2=str(data_actual['Basic Income'][g]).replace(',',''),str(data['Basic Income'][k]).replace(',','')
            basic_income_accuracy,bincome1,bincome2=basic_income_comaprison(bincome1,bincome2)
            
            if config == 'all':
                accuracy_save.append([data_actual['Payslip Number'][g],data['Payslip Number'][k],data_actual['Employer Name'][g],data['Employer Name'][k],fuzzy_comparison_employer_name/100,accuracy_employer_name,employer_name_exist,bleu_employer_name,data_actual['Employee Name'][g],data['Employee Name'][k],fuzzy_comparison_employee_name/100,accuracy_employee_name,employee_name_exist, bleu_employee_name,data_actual['Allowance'][g],data['Allowance'][k],fuzzy_comparison_allowances/100,accuracy_allowances,allowances_exist, bleu_allowances,date1_start,date2_start,accuracy_start_date,date1_end,date2_end,accuracy_end_date,date1_month,date2_month,accuracy_month_date,bincome1,bincome2,basic_income_accuracy])
                dsf=pd.DataFrame(accuracy_save,columns=['Payslip Number Actual','Payslip Number ','Employer Name Actual','Employer Name Extracted','Employer Name Fuzzy Score','Employer Name String Matching Score','Employer Name Ocr Exists','Employer_Bleu_score','Employee Name Actual','Employee Name Extracted', 'Employee Name Fuzzy Score','Employee Name String Matching Score','Employee Name Ocr Exists','Employee_Bleu_score','Allowance Actual','Allowance Extracted','Allowance Fuzzy Score','Allowance String Matching Score','Allowance Ocr Exists','Allowance_Bleu_score','Salary Start date Actual','Salary Start date Extracted','Salary Start date Accuracy','Salary End date Actual','Salary End date Extracted','Salary End date Accuracy','Salary Month Actual','Salary Month Extracted','Salary Month Accuracy','Basic Income Actual','Basic Income Extracted','Basic Income Accuracy'])
            else:
                accuracy_save.append([data_actual['Payslip Number'][g],data['Payslip Number'][k],data_actual['Employer Name'][g],data['Employer Name'][k],fuzzy_comparison_employer_name/100,accuracy_employer_name,employer_name_exist,data_actual['Employee Name'][g],data['Employee Name'][k],fuzzy_comparison_employee_name/100,accuracy_employee_name,employee_name_exist,data_actual['Allowance'][g],data['Allowance'][k],fuzzy_comparison_allowances/100,accuracy_allowances,allowances_exist,date1_start,date2_start,accuracy_start_date,date1_end,date2_end,accuracy_end_date,date1_month,date2_month,accuracy_month_date,bincome1,bincome2,basic_income_accuracy])
                dsf=pd.DataFrame(accuracy_save,columns=['Payslip Number Actual','Payslip Number ','Employer Name Actual','Employer Name Extracted','Employer Name Fuzzy Score','Employer Name String Matching Score','Employer Name Ocr Exists','Employee Name Actual','Employee Name Extracted', 'Employee Name Fuzzy Score','Employee Name String Matching Score','Employee Name Ocr Exists','Allowance Actual','Allowance Extracted','Allowance Fuzzy Score','Allowance String Matching Score','Allowance Ocr Exists','Salary Start date Actual','Salary Start date Extracted','Salary Start date Accuracy','Salary End date Actual','Salary End date Extracted','Salary End date Accuracy','Salary Month Actual','Salary Month Extracted','Salary Month Accuracy','Basic Income Actual','Basic Income Extracted','Basic Income Accuracy'])
dsf.to_excel('results_new_.xlsx',index=False)

C:\Users\afaq.ahmad\.conda\envs\tf_gpu\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\afaq.ahmad\.conda\envs\tf_gpu\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\afaq.ahmad\.conda\envs\tf_gpu\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram